# Импорт библиотек и загрузка данных и предобработка данных для кластерного анализа

### Здесь мы начинаем с импорта необходимых библиотек для работы с данными и кластерным анализом. Важно импортировать pandas для работы с данными в виде датафрейма, а также различные классы и функции из scikit-learn для предобработки данных, кластерного анализа и оценки метрик. Затем мы выбираем числовые признаки из исходного датасета, которые будем использовать для кластеризации. Это важно, так как большинство алгоритмов кластерного анализа работают только с числовыми данными. После выбора признаков мы выполняем стандартизацию данных с помощью StandardScaler. Это необходимо для того, чтобы признаки имели одинаковый масштаб и алгоритмы кластеризации работали корректно.

In [6]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score

df = pd.read_csv('dataset_salary_2024_encoded.csv')

numerical_features = ['work_year', 'salary_in_usd', 'remote_ratio', 'employment_type_encoded', 'company_location_encoded', 'salary_scaled', 'work_year_scaled', 'salary_minmax', 'work_year_minmax', 'salary_normalized', 'work_year_normalized', 'salary_robust_scaled', 'salary_transformed']

X = df[numerical_features]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Обучение и оценка моделей кластерного анализа до понижения размерности

### Здесь мы инициализируем две модели кластеризации: KMeans с 3 кластерами и AgglomerativeClustering с 3 кластерами. Эти модели выбраны для демонстрации различных подходов к кластеризации данных. Для оценки качества кластеризации мы используем три метрики: silhouette_score, davies_bouldin_score и calinski_harabasz_score. Эти метрики помогают понять, насколько хорошо данные кластеризуются.

In [7]:
clustering_models = {
    'KMeans': KMeans(n_clusters=3, random_state=42),  
    'AgglomerativeClustering': AgglomerativeClustering(n_clusters=3)  
}

metrics = {
    'silhouette_score': silhouette_score,
    'davies_bouldin_score': davies_bouldin_score,
    'calinski_harabasz_score': calinski_harabasz_score
}

def evaluate_clustering(models, X):
    results = {}
    for model_name, model in models.items():  
        print(f"Training {model_name}...")  
        labels = model.fit_predict(X) 
        
        model_metrics = {}
        for metric_name, metric_func in metrics.items():  
            score = metric_func(X, labels)  
            model_metrics[metric_name] = score  
        
        results[model_name] = model_metrics  
    return results  

results_before_pca = evaluate_clustering(clustering_models, X_scaled)

print("Metrics before PCA:")
for model_name, model_metrics in results_before_pca.items():  
    print(f"\n{model_name}:")
    for metric_name, score in model_metrics.items():  
        print(f"{metric_name}: {score}")  

Training KMeans...
Training AgglomerativeClustering...
Metrics before PCA:

KMeans:
silhouette_score: 0.3162531261993673
davies_bouldin_score: 1.2624629887868484
calinski_harabasz_score: 3265.861432948947

AgglomerativeClustering:
silhouette_score: 0.2991950939997449
davies_bouldin_score: 1.388139932959817
calinski_harabasz_score: 2768.042791492936


# Понижение размерности данных с помощью PCA

### Здесь мы применяем метод главных компонент (PCA) для уменьшения размерности данных до 2 компонентов. Это делается с целью улучшения работы некоторых алгоритмов кластеризации и визуализации результатов.

In [8]:
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled) 

# Повторное обучение и оценка моделей кластерного анализа после понижения размерности

### После применения PCA мы повторно обучаем и оцениваем те же модели кластерного анализа (KMeans и AgglomerativeClustering) на данных с пониженной размерностью. Мы снова используем те же метрики для оценки качества кластеризации.

In [9]:
results_after_pca = evaluate_clustering(clustering_models, X_pca)

print("Metrics after PCA:")
for model_name, model_metrics in results_after_pca.items():  
    print(f"\n{model_name}:")
    for metric_name, score in model_metrics.items():  
        print(f"{metric_name}: {score}") 

Training KMeans...
Training AgglomerativeClustering...
Metrics after PCA:

KMeans:
silhouette_score: 0.4338178902886084
davies_bouldin_score: 0.932788661817627
calinski_harabasz_score: 6082.9386900143

AgglomerativeClustering:
silhouette_score: 0.3854201059201704
davies_bouldin_score: 0.9331730004724944
calinski_harabasz_score: 5103.215450438755


# Сравнение результатов и выбор наилучшей модели

### В этом блоке мы сравниваем результаты кластеризации до и после применения PCA. Мы анализируем значения метрик (silhouette_score, davies_bouldin_score и calinski_harabasz_score) для каждой модели. Затем мы выбираем наилучшую модель на основе метрики silhouette_score, так как эта метрика дает представление о компактности кластеров и их разделимости.

In [10]:

print("\nComparison of metrics before and after PCA:")
for model_name in results_before_pca.keys():  
    print(f"\n{model_name}:")
    for metric_name in metrics.keys():  
        score_before = results_before_pca[model_name][metric_name]  
        score_after = results_after_pca[model_name][metric_name] 
        print(f"{metric_name}: Before PCA = {score_before}, After PCA = {score_after}")  

best_model_name_before_pca = max(results_before_pca, key=lambda x: results_before_pca[x]['silhouette_score'])
best_model_name_after_pca = max(results_after_pca, key=lambda x: results_after_pca[x]['silhouette_score'])

print(f"\nBest model before PCA: {best_model_name_before_pca} with silhouette_score = {results_before_pca[best_model_name_before_pca]['silhouette_score']}")
print(f"Best model after PCA: {best_model_name_after_pca} with silhouette_score = {results_after_pca[best_model_name_after_pca]['silhouette_score']}")


Comparison of metrics before and after PCA:

KMeans:
silhouette_score: Before PCA = 0.3162531261993673, After PCA = 0.4338178902886084
davies_bouldin_score: Before PCA = 1.2624629887868484, After PCA = 0.932788661817627
calinski_harabasz_score: Before PCA = 3265.861432948947, After PCA = 6082.9386900143

AgglomerativeClustering:
silhouette_score: Before PCA = 0.2991950939997449, After PCA = 0.3854201059201704
davies_bouldin_score: Before PCA = 1.388139932959817, After PCA = 0.9331730004724944
calinski_harabasz_score: Before PCA = 2768.042791492936, After PCA = 5103.215450438755

Best model before PCA: KMeans with silhouette_score = 0.3162531261993673
Best model after PCA: KMeans with silhouette_score = 0.4338178902886084
